In [1]:
import pandas as pd
from sqlalchemy import create_engine

# PROJECT PLAN

Project: merge CPS School Data of 660 schools (from egov.cityofchicago.org) with student demographic composition with a file of 440 affordable housing construction projects from www.johnsnowlabs.com.  We plan to join these on zip code to see if there’s a correlation of neighborhoods with lower income students and affordable housing construction.

ETL: We will join these two CSV’s on zip code.  We will need to aggregate the number of housing units built in a 
specific zip code.  We will need to aggregate schools & student demographic composition per zip code.


In [2]:
schools_csv = "Schools.csv"
schools_df = pd.read_csv(schools_csv)
schools_df.head()

,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name,Long_Name,School_Type,Primary_Category,Is_High_School,Is_Middle_School,Is_Elementary_School,...,Third_Contact_Name,Fourth_Contact_Title,Fourth_Contact_Name,Fifth_Contact_Title,Fifth_Contact_Name,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Location
0,610163,5770,30081,STOCK,Frederick Stock Elementary School,Neighborhood,ES,N,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"7507 W BIRCHWOOD AVE\nChicago, Illinois 60631\..."
1,610558,9598,46611,GOODE HS,Sarah E. Goode STEM Academy,Citywide-Option,HS,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"7651 S HOMAN AVE\nChicago, Illinois 60652\n(41..."
2,609750,1750,49051,SIMPSON HS,Simpson Academy HS for Young Women,Citywide-Option,HS,Y,Y,Y,...,Rita Somen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1321 S PAULINA ST\nChicago, Illinois 60608\n(4..."
3,610571,9636,65015,OMBUDSMAN - WEST HS,Ombudsman Chicago- West,Citywide-Option,HS,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2401 W CONGRESS PKWY\nChicago, Illinois 60612\..."
4,610123,5370,24911,PENN,William Penn Elementary School,Neighborhood,ES,N,Y,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1616 S AVERS AVE\nChicago, Illinois 60623\n(41..."


# We only need "Long Name", "Student_Count_Total", "Student_Count_Low_Income", and "Zip", but we'll select a few more fields
for future analysis.

In [3]:
new_schools_df = schools_df[['Long_Name', 'School_Type', 'Primary_Category','Zip','Student_Count_Total', 'Student_Count_Low_Income','Student_Count_Special_Ed','Student_Count_English_Learners','Student_Count_Black','Student_Count_Hispanic','Student_Count_White','Student_Count_Asian','Student_Count_Native_American', 'Student_Count_Multi','Overall_Rating']].copy()
new_schools_df.head()

,Long_Name,School_Type,Primary_Category,Zip,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Multi,Overall_Rating
0,Frederick Stock Elementary School,Neighborhood,ES,60631,232,37,90,27,1,39,175,16,0,0,Inability to Rate
1,Sarah E. Goode STEM Academy,Citywide-Option,HS,60652,900,788,153,57,459,420,7,2,5,6,Level 1+
2,Simpson Academy HS for Young Women,Citywide-Option,HS,60608,38,37,6,2,28,8,2,0,0,0,Level 2
3,Ombudsman Chicago- West,Citywide-Option,HS,60612,341,320,57,31,187,148,4,0,1,1,Level 2
4,William Penn Elementary School,Neighborhood,ES,60623,311,279,78,13,283,26,1,0,1,0,Level 1+


In [4]:
housing_csv = "Housing.csv"
housing_df = pd.read_csv(housing_csv)
housing_df.head()

,Community_Area_Name,Community_Area_Number,Property_Description,Property_Name,Address,ZIP_Code,Phone_Number,Management_Company,Units,Latitude,Longitude
0,Portage Park,15,ARO,4812-15 W. Montrose Apts.,4812-15 W. Montrose Ave.,60641,630-694-6968,@properties,2,NaN,NaN
1,West Englewood,67,Multifamily,New West Englewood Homes,2109 W. 63rd St.,60636,773-434-4929,Interfaith Housing Corp.,12,NaN,NaN
2,Englewood,68,Multifamily,Antioch Homes II,301 W. Marquette Road,60621,773-994-4546,"Universal Management Service, Inc.",69,41.772564,-87.632419
3,Washington Park,40,Senior HUD 202,St. Edmund's Corners,5556 S. Michigan Ave.,60637,773-667-7583,St. Edmund's Redevelopment Corp.,53,41.792975,-87.622569
4,Humboldt Park,23,Multifamily,Nelson Mandela Apts.,526 N. Troy St.,60624,773-227-6332,Bickerdike Apts.,6,41.891173,-87.705338


# The housing.csv is clean already so we'll go straight to aggregating the number of housing units per zip code and
converting the tuple to a dataframe.

In [13]:
housing_zip = housing_df.groupby(['ZIP_Code'])['Units']
housing_zip_count = housing_zip.sum()
housing_zip_df = pd.DataFrame(housing_zip_count).reset_index()
housing_zip_df = housing_zip_df.rename(columns={"ZIP_Code":"zip",'Units':'units'})
housing_zip_df.head()
# housing_zip_count.dtypes

,zip,units
0,60601,16
1,60605,276
2,60607,233
3,60608,1022
4,60609,1207


# We'll now aggregate the total students and total low-income students by zip code.

In [6]:
students_zip = new_schools_df.groupby(['Zip'])['Student_Count_Total','Student_Count_Low_Income']
students_zip_count = students_zip.sum()
students_zip_count.head()

,Student_Count_Total,Student_Count_Low_Income
Zip,,
60602,1326,1142
60605,2645,975
60607,5477,2358
60608,11009,9798
60609,12972,11467


# We calculate the low-income-percentage-composition of the zip codes and add that percentage to the dataframe

In [7]:
students_zip_count['low_inc_percent'] = 100*students_zip_count['Student_Count_Low_Income']/students_zip_count['Student_Count_Total']
students_zip_count.head()

,Student_Count_Total,Student_Count_Low_Income,low_inc_percent
Zip,,,
60602,1326,1142,86.123680
60605,2645,975,36.862004
60607,5477,2358,43.052766
60608,11009,9798,88.999909
60609,12972,11467,88.398088


# Connect to PostGres

In [8]:
rds_connection_string = "postgres:postgres@localhost:5432/chicago"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
engine.table_names()

['housing', 'schools']

# We reset_index of the students_zip dataframe so we can join to the no-index housing_zip dataframe in SQL.
We also renamed all our fields to lower case since PostGres would automatically make them lower case and this allows 
our dataframe to match our tables in Postgres.

In [10]:
students_zip_count.reset_index(level=0, inplace=True)
students_zip_count = students_zip_count.rename(columns={'Zip':'zip','Student_Count_Total':'student_count_total','Student_Count_Low_Income':'student_count_low_income', 'low_inc_percent':'low_inc_perc'})
students_zip_count.head() 

,zip,student_count_total,student_count_low_income,low_inc_perc
0,60602,1326,1142,86.123680
1,60605,2645,975,36.862004
2,60607,5477,2358,43.052766
3,60608,11009,9798,88.999909
4,60609,12972,11467,88.398088


In [11]:
students_zip_count.to_sql(name='schools', con=engine, if_exists='append', index=False)

In [14]:
housing_zip_df.to_sql(name='housing', con=engine, if_exists='append', index=False)

In [ ]:
students_zip_count.to_sql(name='schools', con=engine, if_exists='append', index=False)

# Here, we query our PostGres tables to confirm that our dataframe exports to sql worked.

In [15]:
pd.read_sql_query('select * from schools', con=engine).head()

,zip,student_count_total,student_count_low_income,low_inc_perc
0,60602,1326,1142,86.123680
1,60605,2645,975,36.862004
2,60607,5477,2358,43.052766
3,60608,11009,9798,88.999909
4,60609,12972,11467,88.398088


In [16]:
pd.read_sql_query('select * from housing', con=engine).head()

,zip,units
0,60601,16
1,60605,276
2,60607,233
3,60608,1022
4,60609,1207


# Our final step is to join the tables on 'zip' and query all the zip codes.

In [19]:
pd.read_sql_query('select * from schools inner join housing on schools.zip = housing.zip order by housing.units desc', con=engine)

,zip,student_count_total,student_count_low_income,low_inc_perc,zip,units
0,60653,5271,4456,84.538038,60653,3071
1,60624,8044,7121,88.525609,60624,1544
2,60612,9016,7676,85.137533,60612,1243
3,60609,12972,11467,88.398088,60609,1207
4,60637,7291,6053,83.020162,60637,1081
5,60608,11009,9798,88.999909,60608,1022
6,60616,7116,5752,80.831928,60616,1007
7,60647,8937,7298,81.660512,60647,909
8,60640,2626,2152,81.949733,60640,827
9,60628,9006,7696,85.454142,60628,758
